# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [106]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [107]:
stocks = pd.read_csv('../sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [108]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2196411664641,
 'week52high': 138.63,
 'week52low': 57.73,
 'week52change': 0.6338826940433482,
 'sharesOutstanding': 17583621542,
 'float': 0,
 'avg10Volume': 98515542,
 'avg30Volume': 110063966,
 'day200MovingAvg': 116.94,
 'day50MovingAvg': 133.2,
 'employees': 141528,
 'ttmEPS': 3.39,
 'ttmDividendRate': 0.837285819781181,
 'dividendYield': 0.006456721643941801,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-23',
 'nextEarningsDate': '0',
 'peRatio': 38.35669006155207,
 'beta': 1.2098910928219326,
 'maxChangePercent': 48.61903979134156,
 'year5ChangePercent': 4.704689082812013,
 'year2ChangePercent': 2.401013236511807,
 'year1ChangePercent': 0.6334678736969794,
 'ytdChangePercent': -0.04342688414344228,
 'month6ChangePercent': 0.3301996978706848,
 'month3ChangePercent': 0.07143415636549892,
 'month1ChangePercent': 0.003949877565975677,
 'day30ChangePercent': 0.003904366247123892,
 'day5ChangePercent': -0.01473709412602334}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [109]:
data['year1ChangePercent']

0.6334678736969794

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [110]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [111]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.787,0.424825,N/A
1,AAL,16.730,-0.462945,N/A
2,AAP,172.387,0.104784,N/A
3,AAPL,132.230,0.625961,N/A
4,ABBV,113.150,0.325492,N/A
...,...,...,...,...
500,YUM,110.500,0.0325419,N/A
501,ZBH,166.740,0.083468,N/A
502,ZBRA,427.660,0.637484,N/A
503,ZION,51.897,0.00523079,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [112]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,41.821,2.383,N/A
1,179,FCX,31.847,1.37146,N/A
2,275,LB,47.259,1.32346,N/A
3,23,ALB,189.500,1.27034,N/A
4,345,NVDA,526.560,1.11548,N/A
5,387,PYPL,245.400,1.09489,N/A
6,24,ALGN,571.290,1.07101,N/A
7,490,WST,304.036,0.917696,N/A
8,385,PWR,78.190,0.862545,N/A
9,85,CDNS,140.100,0.850511,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [113]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")

    while True:
        try:
            val = float(portfolio_size)
            print(val)
            break
        except ValueError:
            print('Please enter a number.')
            portfolio_size = input('Enter the size of your portfolio: ')
            

In [114]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,41.821,2.383,4782
1,179,FCX,31.847,1.37146,6280
2,275,LB,47.259,1.32346,4231
3,23,ALB,189.500,1.27034,1055
4,345,NVDA,526.560,1.11548,379
5,387,PYPL,245.400,1.09489,814
6,24,ALGN,571.290,1.07101,350
7,490,WST,304.036,0.917696,657
8,385,PWR,78.190,0.862545,2557
9,85,CDNS,140.100,0.850511,1427


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [115]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.783,N/A,0.423859,N/A,0.353057,N/A,0.192188,N/A,0.0607527,N/A,N/A
1,AAL,16.620,N/A,-0.464752,N/A,0.336247,N/A,0.275924,N/A,-0.0468828,N/A,N/A
2,AAP,169.583,N/A,0.107431,N/A,0.146873,N/A,0.069621,N/A,0.0344051,N/A,N/A
3,AAPL,132.410,N/A,0.633151,N/A,0.327099,N/A,0.0720338,N/A,0.00400465,N/A,N/A
4,ABBV,115.730,N/A,0.326077,N/A,0.125964,N/A,0.301002,N/A,0.070972,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.630,N/A,0.0317858,N/A,0.195692,N/A,0.08837,N/A,-0.0299767,N/A,N/A
501,ZBH,167.150,N/A,0.0820493,N/A,0.19341,N/A,0.094291,N/A,0.0576129,N/A,N/A
502,ZBRA,420.830,N/A,0.640872,N/A,0.508333,N/A,0.398758,N/A,0.0971875,N/A,N/A
503,ZION,49.505,N/A,0.00514221,N/A,0.547983,N/A,0.593675,N/A,0.200692,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [116]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]


for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0


for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100



#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.783,N/A,0.423859,0.885149,0.353057,0.70297,0.192188,0.633663,0.0607527,0.671287,N/A
1,AAL,16.620,N/A,-0.464752,0.0158416,0.336247,0.683168,0.275924,0.736634,-0.0468828,0.120792,N/A
2,AAP,169.583,N/A,0.107431,0.554455,0.146873,0.388119,0.069621,0.372277,0.0344051,0.544554,N/A
3,AAPL,132.410,N/A,0.633151,0.948515,0.327099,0.671287,0.0720338,0.380198,0.00400465,0.364356,N/A
4,ABBV,115.730,N/A,0.326077,0.813861,0.125964,0.334653,0.301002,0.762376,0.070972,0.70099,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.630,N/A,0.0317858,0.423762,0.195692,0.477228,0.08837,0.421782,-0.0299767,0.184158,N/A
501,ZBH,167.150,N/A,0.0820493,0.516832,0.19341,0.473267,0.094291,0.439604,0.0576129,0.651485,N/A
502,ZBRA,420.830,N/A,0.640872,0.950495,0.508333,0.857426,0.398758,0.853465,0.0971875,0.8,N/A
503,ZION,49.505,N/A,0.00514221,0.364356,0.547983,0.879208,0.593675,0.948515,0.200692,0.946535,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [117]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.783,N/A,0.423859,0.885149,0.353057,0.70297,0.192188,0.633663,0.0607527,0.671287,0.723267
1,AAL,16.620,N/A,-0.464752,0.0158416,0.336247,0.683168,0.275924,0.736634,-0.0468828,0.120792,0.389109
2,AAP,169.583,N/A,0.107431,0.554455,0.146873,0.388119,0.069621,0.372277,0.0344051,0.544554,0.464851
3,AAPL,132.410,N/A,0.633151,0.948515,0.327099,0.671287,0.0720338,0.380198,0.00400465,0.364356,0.591089
4,ABBV,115.730,N/A,0.326077,0.813861,0.125964,0.334653,0.301002,0.762376,0.070972,0.70099,0.65297
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.630,N/A,0.0317858,0.423762,0.195692,0.477228,0.08837,0.421782,-0.0299767,0.184158,0.376733
501,ZBH,167.150,N/A,0.0820493,0.516832,0.19341,0.473267,0.094291,0.439604,0.0576129,0.651485,0.520297
502,ZBRA,420.830,N/A,0.640872,0.950495,0.508333,0.857426,0.398758,0.853465,0.0971875,0.8,0.865347
503,ZION,49.505,N/A,0.00514221,0.364356,0.547983,0.879208,0.593675,0.948515,0.200692,0.946535,0.784653


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [118]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,184.350,N/A,1.27099,0.994059,1.07168,0.988119,0.972198,0.988119,0.298021,0.994059,0.991089
1,FCX,32.861,N/A,1.3861,0.99802,1.26139,0.99604,0.770639,0.982178,0.236158,0.978218,0.988614
2,SIVB,471.440,N/A,0.824317,0.976238,1.13862,0.992079,0.706413,0.970297,0.301475,0.99604,0.983663
3,AMAT,109.438,N/A,0.685439,0.958416,0.672857,0.928713,0.663981,0.962376,0.200818,0.948515,0.949505
4,LB,46.763,N/A,1.27591,0.99604,1.45744,0.99802,0.374675,0.827723,0.178223,0.936634,0.939604
5,GM,54.910,N/A,0.423349,0.883168,0.929833,0.978218,0.510208,0.914851,0.220088,0.964356,0.935149
6,LRCX,591.670,N/A,0.84205,0.980198,0.607155,0.90297,0.527808,0.924752,0.141325,0.906931,0.928713
7,MOS,28.789,N/A,0.336405,0.823762,1.17438,0.994059,0.517476,0.920792,0.230904,0.972277,0.927723
8,APTV,145.110,N/A,0.578211,0.928713,0.781194,0.958416,0.463701,0.889109,0.154754,0.916832,0.923267
9,KLAC,315.500,N/A,0.73764,0.970297,0.55725,0.889109,0.487616,0.90297,0.160899,0.924752,0.921782


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [119]:
portfolio_input()

10000000.0


In [120]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,184.350,1084,1.27099,0.994059,1.07168,0.988119,0.972198,0.988119,0.298021,0.994059,0.991089
1,FCX,32.861,6086,1.3861,0.99802,1.26139,0.99604,0.770639,0.982178,0.236158,0.978218,0.988614
2,SIVB,471.440,424,0.824317,0.976238,1.13862,0.992079,0.706413,0.970297,0.301475,0.99604,0.983663
3,AMAT,109.438,1827,0.685439,0.958416,0.672857,0.928713,0.663981,0.962376,0.200818,0.948515,0.949505
4,LB,46.763,4276,1.27591,0.99604,1.45744,0.99802,0.374675,0.827723,0.178223,0.936634,0.939604
5,GM,54.910,3642,0.423349,0.883168,0.929833,0.978218,0.510208,0.914851,0.220088,0.964356,0.935149
6,LRCX,591.670,338,0.84205,0.980198,0.607155,0.90297,0.527808,0.924752,0.141325,0.906931,0.928713
7,MOS,28.789,6947,0.336405,0.823762,1.17438,0.994059,0.517476,0.920792,0.230904,0.972277,0.927723
8,APTV,145.110,1378,0.578211,0.928713,0.781194,0.958416,0.463701,0.889109,0.154754,0.916832,0.923267
9,KLAC,315.500,633,0.73764,0.970297,0.55725,0.889109,0.487616,0.90297,0.160899,0.924752,0.921782


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [121]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [122]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [123]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [124]:
writer.save()